# Pain Data Preparation
This notebook prepares the pain dataset in to be able to successfully train a convolutional neural network. Data augmentation techniques such as greyscaling, histogram equalization, etc. are employed.

In [1]:
# Relevant imports
import os
import sys
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import math

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from Scripts import Data_Loader_Functions as DL
from Scripts import Image_Processor as IP

In [2]:
# Define folder paths
RAW_DATA = os.path.join(module_path, "Data", "Raw Data", "Pain")
PREPROCESSED_DATA = os.path.join(module_path, "Data", "Preprocessed Data", "Pain")
AUGMENTED_DATA = os.path.join(module_path, "Data", "Augmented Data", "Pain")
AUGMENTED_DATA_TWOSTEP = os.path.join(module_path, "Data", "Augmented Data", "Pain Two-Step Augmentation")

## Create Folder Structure
First, we will duplicate the folder structure in "Raw Data" into "Preprocessed Data" and "Augmented Data".

In [ ]:
# Duplicate folder structure
DL.mirror_folder_structure(RAW_DATA, PREPROCESSED_DATA)
DL.mirror_folder_structure(RAW_DATA, AUGMENTED_DATA)
DL.mirror_folder_structure(RAW_DATA, AUGMENTED_DATA_TWOSTEP)

## Explore Data

In [ ]:
# Get original pain distribution
img_paths = np.array(DL.get_image_paths(RAW_DATA))
labels = np.array(DL.get_labels(img_paths))
no_pain_labels = labels[labels[:,4].astype(int)==0]
pain_labels = labels[labels[:,4].astype(int)>0]
print("Pain Labels:", len(pain_labels))
print("No Pain Labels:", len(no_pain_labels))

In [ ]:
# Get number of clients per group
g1_img_paths = [x for x in os.listdir(os.path.join(RAW_DATA, "group_1")) if x != '.DS_Store']
g2_img_paths = [x for x in os.listdir(os.path.join(RAW_DATA, "group_2")) if x != '.DS_Store']
print("Group 1:", len(g1_img_paths))
print("Group 2:", len(g2_img_paths))

In [ ]:
# Get number of sessions per client
g1_img_paths = np.array(DL.get_image_paths(os.path.join(RAW_DATA, "group_1")))
g2_img_paths = np.array(DL.get_image_paths(os.path.join(RAW_DATA, "group_2")))
g1_labels = np.array(DL.get_labels(g1_img_paths))
g2_labels = np.array(DL.get_labels(g2_img_paths))
df_1 = pd.DataFrame(g1_labels, columns=['Person','Session','Culture','Frame','Pain']).astype(int)
df_2 = pd.DataFrame(g2_labels, columns=['Person','Session','Culture','Frame','Pain']).astype(int)
df_1['Group'] = 1
df_2['Group'] = 2
df = pd.concat([df_1, df_2])
sess_num = pd.DataFrame(df.groupby(['Person', 'Group'])['Session'].nunique()).sort_values(['Group','Person'])
sess_num

In [ ]:
# Average number of sessions per group
print("Average Sessions Group 1: {0:.2f}".format(df_1.groupby('Person')['Session'].nunique().mean()))
print("Average Sessions Group 2: {0:.2f}".format(df_2.groupby('Person')['Session'].nunique().mean()))

In [ ]:
# Pain / No Pain per group
print("Group 1 Pain/No Pain/Ratio: {} | {}".format(df[(df['Group'] == 1) & (df['Pain'] == 0)].count()[0], df[(df['Group'] == 1) & (df['Pain'] > 0)].count()[0]))
print("Group 2 Pain/No Pain/Ratio: {} | {}".format(df[(df['Group'] == 2) & (df['Pain'] == 0)].count()[0], df[(df['Group'] == 2) & (df['Pain'] > 0)].count()[0]))

## Process Images
We will now process the images. Preprocessing includes converting to greyscale, and histogram equalization.

In [ ]:
# Preprocess images
IP.bulk_process_images(RAW_DATA, PREPROCESSED_DATA, ".jpg")

In [ ]:
# Flip images and copy originals into augmented data folder
IP.bulk_augment_images(PREPROCESSED_DATA, AUGMENTED_DATA_TWOSTEP, ".jpg", "flip", "pain", label_threshold=-1)
IP.bulk_augment_images(PREPROCESSED_DATA, AUGMENTED_DATA_TWOSTEP, ".jpg", "original", "pain", label_threshold=-1)

In [ ]:
# Rotate Originals and flipped images, and ensure that naming conventions stay consistent
IP.bulk_augment_images(AUGMENTED_DATA_TWOSTEP, AUGMENTED_DATA_TWOSTEP, "_flipped.jpg", "rotate_crop", "pain", label_threshold=-1)
IP.bulk_augment_images(AUGMENTED_DATA_TWOSTEP, AUGMENTED_DATA_TWOSTEP, "_original.jpg", "rotate_crop", "pain", label_threshold=-1)
IP.bulk_rename_files(AUGMENTED_DATA_TWOSTEP, AUGMENTED_DATA_TWOSTEP, "_rotated", "_straight")

In [ ]:
# Crop images to same maximum width and height (10-degree rotation in previous step cropped rotated images 
# down to (215, 215), so this is chosen as a max width/height)
IP.bulk_crop_images(AUGMENTED_DATA_TWOSTEP, AUGMENTED_DATA_TWOSTEP, (215, 215))

In [ ]:
# Downsample augmented data
DL.downsample_data(os.path.join(AUGMENTED_DATA_TWOSTEP, "group_1"))
DL.downsample_data(os.path.join(AUGMENTED_DATA_TWOSTEP, "group_2"))

## Split Group 2 Data
Split Group 2 Data into 40% Test Data and an additional 60% Test Data.

In [12]:
# Create DataFrame to enable filtering by labels
img_paths = DL.get_image_paths(os.path.join(AUGMENTED_DATA_TWOSTEP, 'group_2'))
labels = DL.get_labels(img_paths)
df = pd.DataFrame(labels, columns=['Person','Session','Culture','Frame','Pain', 'Trans_1', 'Trans_2'])
df[['Person','Session','Culture','Frame','Pain']] = df[['Person','Session','Culture','Frame','Pain']].astype(int)
df['img_path'] = img_paths
df[['Trans_1', 'Trans_2', 'img_path']] = df[['Trans_1', 'Trans_2', 'img_path']].astype(str)

In [13]:
# Split Dataset into Train and Test
origin_path = os.path.join(AUGMENTED_DATA_TWOSTEP, 'group_2')
train_path = os.path.join(AUGMENTED_DATA_TWOSTEP, 'group_2_train')
test_path = os.path.join(AUGMENTED_DATA_TWOSTEP, 'group_2_test')
DL.move_train_test_data(df, origin_path, train_path, test_path)

In [16]:
# Verify that split is 40 / 60
img_train = DL.get_image_paths(train_path)
img_test = DL.get_image_paths(test_path)
print("Test: {0:.2f}".format(len(img_test) / (len(img_test) + len(img_train))))
print("Train: {0:.2f}".format(len(img_train) / (len(img_test) + len(img_train))))

Test: 0.40
Train: 0.60


## Split Dataset (Randomly)
Splitting the dataset into training data and test data, by sampling without replacement from the train_data

In [ ]:
# Get all images and select 20% at random
img_paths = DL.get_image_paths(AUGMENTED_DATA_TWOSTEP)
np.random.shuffle(img_paths)
split_idx = int(len(img_paths)*0.2)
img_paths_test = img_paths[:split_idx]

In [ ]:
# Ensure that each client is represented with ~20% in the test data set
img_per_client_test = np.unique(np.array(DL.get_labels(img_paths_test))[:,0], return_counts=True)[1]
img_per_client_total = np.unique(np.array(DL.get_labels(img_paths))[:,0], return_counts=True)[1]
img_per_client_test / img_per_client_total

In [ ]:
# Ensure that the test set is balanced
pain = np.array(img_paths_test)[np.array(DL.get_labels(img_paths_test))[:,4].astype(int) >= 1]
pain_test_labels = np.array(DL.get_labels(pain))
all_test_labels = np.array(DL.get_labels(img_paths_test))
print("Test Pain Split:",len(pain_labels) / len(all_labels))

In [ ]:
# Investigate the split for each client in the test set
DL.print_pain_split_per_client(all_test_labels)

In [ ]:
# Move test data set into test folder
for src in img_paths_test:
    file = os.path.basename(src)
    dest = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(src))), 'test')
    if not os.path.isdir(dest):
        os.mkdir(dest)
    try:
        os.rename(src, os.path.join(dest, file))
    except FileNotFoundError:
        pass